In [40]:
import numpy as np
import pandas as pd

импорт функции считающей частоты синонимичных кодонов

In [41]:
from rel_freq import relfreq

GC3 - доля G/C в третьих позициях кодонов
rawdata - название исходного файла из базы данных CCUG
result -  имя файла куда будет сохранена csv таблица абсолютных частот встречаемости кодонов и GC3 к колонке gc_fraction
normalized_result - имя файла куда будет сохранена csv таблица с частотами встречаемости синонимичных кодонов - gc usage frequency (частоты кодонов посчитанные для группы синонимичных)

In [42]:
rawdata = "gbpln.txt"
result = "df_plant.csv"
normalized_result = "df_plant_relfreq.csv"

следующая функция (gc3) возвращает частоту кодонов заканчивающихся на G или C - GC3, суммируя частоты таких кодонов

In [43]:
def gc3(org):
    org=dict(org)
    gc = 0
    for key in org.keys():
        if key[2] == "G" or key[2] == "C":
            gc+=org[key]
    return(gc)

index - файл содержащий порядок кодонов, в котором записаны их количества для каждого элемента датасета в базе данных CCUG. Соответствует файлу CODON_LABEL из базы данных CCUG. В следующей ячейке создается список кодонов используемых для названия колонок в будущем датафрейме

In [44]:
with open("index","r") as infile:
    infile.readline()
    col = infile.readline().strip("\n").split(" ")

Открываем файл (типа .spsum из CCUG) с количеством кодонов для разных организмов, cоздаем датафрейм c колонкой GC3 - gc_fraction 

In [45]:
data = []
with open(rawdata,"r") as f:
    for line in f:
        if not ":" in line:
            data.append(np.array(line.strip("\n").strip(" ").split(" ")).astype("int64"))
df = pd.DataFrame(data, columns = col)

создаем датафрейм с частотами кодонов внутри группы синонимичных при помощи функции relfreq

In [46]:
normalized_df={}
for ind in df.index:
    normalized_df[ind] = relfreq(df.loc[ind].to_dict())
normalized_df = pd.DataFrame(normalized_df).T

находим сумму всех кодонов и заменяем количество кодона на абсолютную частоту его встречаемости

In [47]:
df["sum"] = (df.sum(axis=1))
for elem in col:
    df[elem] = df[elem]/df["sum"]
df = df.drop(["sum"], axis=1)

считаем GC3 и добавляем его в оба датасета

In [48]:
gc = []
for ind in range(len(df)):
    gc.append(gc3(df.iloc[ind]))
df["gc_fraction"] = gc
normalized_df["gc_fraction"] = df["gc_fraction"]

сортируем элементы датафреймoв согласно GC3 по возрастанию и сохраняем в соответствующие файлы

In [49]:
df = df.sort_values(by = "gc_fraction")
normalized_df = normalized_df.sort_values(by = "gc_fraction")
df.to_csv(result)
normalized_df.to_csv(normalized_result)